In [ ]:
class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler, Sampler, SubsetRandomSampler
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models
import matplotlib.pyplot as plt
import time
import os
import copy

import albumentations as A
from albumentations.pytorch import ToTensorV2

from math import ceil
import seaborn as sn

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, classification_report, balanced_accuracy_score

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f'{bcolors.OK}{device}{bcolors.RESET}')

In [ ]:
# set the attempt number in attempt
# # set the model name in model
# set the model version in version

hyper_parameter = {
    "attempt": 'image200-custom-sampler',
    "model": 'efficientnet',
    "version": 'b0',
    "learning_rate": 0.01,
    "batch_size": 8,
    "num_workers": 2,
    "no_epochs": 10,
    "image_size": 224, 
    "in_channels": 3, 
    "num_classes": 8,
    "load_model": True
}

In [ ]:
# set data directory
data_dir = '../Data/ISIC2019/images200/'
sets = ['train', 'test']
# to use checkpoint saving create a directory named "checkpoint" and a sub directory in the name of the model
checkpoint_path = f'./checkpoints/{hyper_parameter["model"]}/{hyper_parameter["model"]}{hyper_parameter["version"]}-{hyper_parameter["attempt"]}'


print(f'{bcolors.OK}Enviroment setup complete 😊🐼!{bcolors.RESET}')

In [ ]:
classes = ['AK', 'BCC', 'BKL', 'DF', 'MEL', 'NV', 'SCC', 'VASC']

train_class_frequency = []
for classname in iter(classes):
    train_class_frequency.append(len(os.listdir(os.path.join(data_dir, 'train', classname))))

image_frequency = pd.DataFrame(columns=sets, index=classes)
for dataset in iter(sets):
    for classname in iter(classes):
        image_frequency[dataset][classname] = len(os.listdir(os.path.join(data_dir, dataset, classname)))
image_frequency.index = image_frequency.index.rename('classes')

print(image_frequency.to_markdown())
print(image_frequency.sum(axis=0))

image_ratio = pd.DataFrame(columns=sets, index=classes)
for dataset in iter(sets):
    for classname in iter(classes):
        image_ratio[dataset][classname] = (image_frequency[dataset][classname] / image_frequency[dataset].sum())*100
image_ratio.index = image_frequency.index.rename('classes')
print(image_ratio.to_markdown())

In [ ]:
class Transforms:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img, *args, **kwargs):
        return self.transforms(image=np.array(img))['image']

In [ ]:
# set the mean and std based on the model documentation
# for efficient net bo - pytorch documentation
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
# hyper_parameter['image_size']=32

data_transforms = {
    'train': A.Compose([A.Resize(hyper_parameter['image_size'] , hyper_parameter['image_size']), A.Normalize(mean, std), ToTensorV2()]),
    'test': A.Compose([A.Resize(hyper_parameter['image_size'] , hyper_parameter['image_size']), A.Normalize(mean, std), ToTensorV2()])
}

In [ ]:
# create dataset and dataloader
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), transform =Transforms(transforms=data_transforms[x])) for x in sets}

In [ ]:
# ratio_of=hyper_parameter['batch_size']
ratio_of = len(image_datasets['train'])
# if we consider trainset (0.80) to be 10,360 the total dataset 12,950 (1.00)
# test would be 2,590 (0.20)
# ratio_of = 400
class_ratio=0.125
targets = np.array(image_datasets['train'].targets)
ak_idxs = np.where(targets==0)[0]
bcc_idxs = np.where(targets==1)[0]
bkl_idxs = np.where(targets==2)[0]
df_idxs = np.where(targets==3)[0]
mel_idxs = np.where(targets==4)[0]
nv_idxs = np.where(targets==5)[0]
scc_idxs = np.where(targets==6)[0]
vasc_idxs = np.where(targets==7)[0]
ak = np.random.choice(ak_idxs, int(ratio_of * class_ratio), replace=True)
bcc = np.random.choice(bcc_idxs, int(ratio_of * class_ratio), replace=True)
bkl = np.random.choice(bkl_idxs, int(ratio_of * class_ratio), replace=True)
df = np.random.choice(df_idxs, int(ratio_of * class_ratio), replace=True)
mel = np.random.choice(mel_idxs, int(ratio_of * class_ratio), replace=False)
nv = np.random.choice(nv_idxs, int(ratio_of * class_ratio), replace=True)
scc = np.random.choice(scc_idxs, int(ratio_of * class_ratio), replace=True)
vasc = np.random.choice(vasc_idxs, int(ratio_of * class_ratio), replace=True)
idxs = np.hstack([ak, bcc, bkl, df, mel, nv, scc, vasc])
# np.random.shuffle(idxs)
np.random.shuffle([ak, bcc, bkl, df, mel, nv, scc, vasc])

print(len(ak))
print(len(idxs)==len(ak)*8)

In [ ]:
class_counts = train_class_frequency
class_weights = np.zeros_like(len(image_datasets['train']), dtype=np.float16)
median_freq = np.median(class_counts)
print(median_freq)
class_weights = [median_freq/c for c in class_counts]
print(class_weights)

sample_weights = [0] * len(image_datasets['train'])
for idx, (data, label) in enumerate(image_datasets['train']):
    class_weight = class_weights[label]
    sample_weights[idx] = class_weight

medianSampler = WeightedRandomSampler(sample_weights, sum(train_class_frequency))

for w in class_weights:
    print(f'{(w*100):2f}')

1745.5
[2.013264129181084, 0.5252783629250677, 0.6652057926829268, 7.303347280334728, 0.3860017691287041, 0.1355728155339806, 2.7794585987261144, 6.899209486166008]


In [ ]:
# ensure an equal number of representatives from each class in each batch
# The class_counts variable is a list that contains the number of samples in each class. 
# The weights variable is then created by taking the reciprocal of each class count and normalizing the values so that they sum to 1. 
# This gives each class an equal probability of being selected. Then the WeightedRandomSampler is instantiated with the weights and the number of samples. 
# Finally, the DataLoader is instantiated with the dataset, the desired batch size, and the sampler.

class_counts = train_class_frequency # list of number of samples per class
weights = [1/c for c in class_counts]
weights = [w/sum(weights) for w in weights]
sample_weights = [0] * len(image_datasets['train'])

for idx, (data, label) in enumerate(image_datasets['train']):
    class_weight = weights[label]
    sample_weights[idx] = class_weight

weightedSampler = WeightedRandomSampler(sample_weights, sum(train_class_frequency))

for w in weights:
    print(f'{(w*100):2f}')

9.722467
2.536677
3.212416
35.269368
1.864082
0.654709
13.422578
33.317703


In [ ]:
# oversampling
# setting weights to each sample based on class population
class_weights = []
for root, subdir, files in os.walk(os.path.join(data_dir, "test")):
    if len(files) > 0:
        class_weights.append(1/len(files))

sample_weights = [0] * len(image_datasets['test'])

for idx, (data, label) in enumerate(image_datasets['test']):
    class_weight = class_weights[label]
    sample_weights[idx] = class_weight

sampler = WeightedRandomSampler(sample_weights, num_samples=len(sample_weights), replacement=True)
class_weights

In [ ]:
# if we consider trainset (0.80) to be 10,360 the total dataset 12,950 (1.00)
# test would be 2,590 (0.20)
class RatioSampler(Sampler):
    def __init__(self, dataset, flags, is_test=False, class_ratio=0.125):
        self.dataset = dataset
        if(is_test):
            # self.n = 2590
            self.n = 400
        else:
            self.n = len(dataset)
        self.class_ratio = class_ratio
        self.flags = flags
    
    def __iter__(self):
        targets = np.array(self.dataset.targets)
        ak_idxs = np.where(targets==0)[0]
        bcc_idxs = np.where(targets==1)[0]
        bkl_idxs = np.where(targets==2)[0]
        df_idxs = np.where(targets==3)[0]
        mel_idxs = np.where(targets==4)[0]
        nv_idxs = np.where(targets==5)[0]
        scc_idxs = np.where(targets==6)[0]
        vasc_idxs = np.where(targets==7)[0]
        ak = np.random.choice(ak_idxs, int(self.n * self.class_ratio), replace=self.flags[0])
        bcc = np.random.choice(bcc_idxs, int(self.n * self.class_ratio), replace=self.flags[1])
        bkl = np.random.choice(bkl_idxs, int(self.n * self.class_ratio), replace=self.flags[2])
        df = np.random.choice(df_idxs, int(self.n * self.class_ratio), replace=self.flags[3])
        mel = np.random.choice(mel_idxs, int(self.n * self.class_ratio), replace=self.flags[4])
        nv = np.random.choice(nv_idxs, int(self.n * self.class_ratio), replace=self.flags[5])
        scc = np.random.choice(scc_idxs, int(self.n * self.class_ratio), replace=self.flags[6])
        vasc = np.random.choice(vasc_idxs, int(self.n * self.class_ratio), replace=self.flags[7])
        idxs = np.hstack([ak, bcc, bkl, df, mel, nv, scc, vasc])
        np.random.shuffle(idxs)
        return iter(idxs)

    def __len__(self):
        return self.n

In [ ]:
class EqualDataLoader(Dataset):
    def __init__(self, dataset, class_counts):
        self.dataset = dataset
        self.class_counts = class_counts
        self.class_samples = []
        for i in range(len(class_counts)):
            self.class_samples.append([j for j in range(len(dataset)) if dataset[j][1] == i])

    def __getitem__(self, index):
        class_idx = index % len(self.class_counts)
        sample_idx = np.random.choice(self.class_samples[class_idx])
        return self.dataset[sample_idx]

    def __len__(self):
        return sum(self.class_counts)

In [ ]:
# dataloaders = {x: DataLoader(image_datasets[x], batch_size=hyper_parameter['batch_size'], shuffle=True) for x in sets}

# train_flags = [True, True, True, True, True, False, True, True]
# test_flags = [True, True, True, True, True, False, True, True]

# dataloaders = {
#     'train': DataLoader(image_datasets['train'], batch_size=hyper_parameter['batch_size'], sampler=RatioSampler(dataset=image_datasets['train'], flags=train_flags)),
#     'test': DataLoader(image_datasets['test'], batch_size=hyper_parameter['batch_size'], sampler=RatioSampler(image_datasets['test'], flags=test_flags, is_test=True))
# }

# dataloaders = {
#     'train': DataLoader(image_datasets['train'], batch_size=hyper_parameter['batch_size'], sampler=RatioSampler(image_datasets['train'], flags=train_flags)),
#     'test': DataLoader(image_datasets['test'], batch_size=hyper_parameter['batch_size'], sampler=RatioSampler(image_datasets['test'], flags=test_flags))
# }

dataloaders = {
    'train': DataLoader(EqualDataLoader(dataset=image_datasets['train'], class_counts=train_class_frequency), batch_size=hyper_parameter['batch_size'], sampler=equalitySampler),
    'test': DataLoader(image_datasets['test'], batch_size=hyper_parameter['batch_size'], shuffle=True)
}

dataset_sizes = {x: len(image_datasets[x]) for x in sets}
class_names = image_datasets['train'].classes

print('train: {} test: {}'.format(dataset_sizes['train'], dataset_sizes['test']))
print(class_names)

In [ ]:
# Train Loader - count the frequency of images in every iteration
num_ak, num_bcc, num_bkl, num_df, num_mel, num_nv, num_scc, num_vasc = 0,0,0,0,0,0,0,0
for data, labels in dataloaders['train']:
    num_ak += torch.sum(labels==0)
    num_bcc += torch.sum(labels==1)
    num_bkl += torch.sum(labels==2)
    num_df += torch.sum(labels==3)
    num_mel += torch.sum(labels==4)
    num_nv += torch.sum(labels==5)
    num_scc += torch.sum(labels==6)
    num_vasc += torch.sum(labels==7)
print(f'AK: {num_ak}, BCC: {num_bcc}, BKL: {num_bkl}, DF: {num_df}, MEL: {num_mel}, NV: {num_nv}, SCC: {num_scc}, VASC: {num_vasc}')

In [ ]:
# Test Loader - count the frequency of images in every iteration
num_ak, num_bcc, num_bkl, num_df, num_mel, num_nv, num_scc, num_vasc = 0,0,0,0,0,0,0,0
for data, labels in dataloaders['test']:
    num_ak += torch.sum(labels==0)
    num_bcc += torch.sum(labels==1)
    num_bkl += torch.sum(labels==2)
    num_df += torch.sum(labels==3)
    num_mel += torch.sum(labels==4)
    num_nv += torch.sum(labels==5)
    num_scc += torch.sum(labels==6)
    num_vasc += torch.sum(labels==7)
print(f'AK: {num_ak}, BCC: {num_bcc}, BKL: {num_bkl}, DF: {num_df}, MEL: {num_mel}, NV: {num_nv}, SCC: {num_scc}, VASC: {num_vasc}')

In [ ]:
# Helper function for inline image display
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

images, classes = next(iter(dataloaders['train']))

# Create a grid from the images and show them
img_grid = torchvision.utils.make_grid(images)
imshow(img_grid, title=[class_names[x] for x in classes])

In [ ]:
# class Identity(nn.Module):
#     def __init__(self):
#         super(Identity, self).__init__()
    
#     def forward(self, x):
#         return x

In [ ]:
base_model = [
    # expand_ratio, channels, repeats, stride, kernel_size
    [1, 16, 1, 1, 3],
    [6, 24, 2, 2, 3],
    [6, 40, 2, 2, 5],
    [6, 80, 3, 2, 3],
    [6, 112, 3, 1, 5],
    [6, 192, 4, 2, 5],
    [6, 320, 1, 1, 3],
]

phi_values = {
    # tuple of: (phi_value, resolution, drop_rate)
    "b0": (0, 224, 0.2),  # alpha, beta, gamma, depth = alpha ** phi
    "b1": (0.5, 240, 0.2),
    "b2": (1, 260, 0.3),
    "b3": (2, 300, 0.3),
    "b4": (3, 380, 0.4),
    "b5": (4, 456, 0.4),
    "b6": (5, 528, 0.5),
    "b7": (6, 600, 0.5),
}

class CNNBlock(nn.Module):
    def __init__(
            self, in_channels, out_channels, kernel_size, stride, padding, groups=1
    ):
        super(CNNBlock, self).__init__()
        self.cnn = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            groups=groups,
            bias=False,
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.silu = nn.SiLU() # SiLU <-> Swish

    def forward(self, x):
        return self.silu(self.bn(self.cnn(x)))

class SqueezeExcitation(nn.Module):
    def __init__(self, in_channels, reduced_dim):
        super(SqueezeExcitation, self).__init__()
        self.se = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # C x H x W -> C x 1 x 1
            nn.Conv2d(in_channels, reduced_dim, 1),
            nn.SiLU(),
            nn.Conv2d(reduced_dim, in_channels, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return x * self.se(x)

class InvertedResidualBlock(nn.Module):
    def __init__(
            self,
            in_channels,
            out_channels,
            kernel_size,
            stride,
            padding,
            expand_ratio,
            reduction=4, # squeeze excitation
            survival_prob=0.8, # for stochastic depth
    ):
        super(InvertedResidualBlock, self).__init__()
        self.survival_prob = 0.8
        self.use_residual = in_channels == out_channels and stride == 1
        hidden_dim = in_channels * expand_ratio
        self.expand = in_channels != hidden_dim
        reduced_dim = int(in_channels / reduction)

        if self.expand:
            self.expand_conv = CNNBlock(
                in_channels, hidden_dim, kernel_size=3, stride=1, padding=1,
            )

        self.conv = nn.Sequential(
            CNNBlock(
                hidden_dim, hidden_dim, kernel_size, stride, padding, groups=hidden_dim,
            ),
            SqueezeExcitation(hidden_dim, reduced_dim),
            nn.Conv2d(hidden_dim, out_channels, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

    def stochastic_depth(self, x):
        if not self.training:
            return x

        binary_tensor = torch.rand(x.shape[0], 1, 1, 1, device=x.device) < self.survival_prob
        return torch.div(x, self.survival_prob) * binary_tensor

    def forward(self, inputs):
        x = self.expand_conv(inputs) if self.expand else inputs

        if self.use_residual:
            return self.stochastic_depth(self.conv(x)) + inputs
        else:
            return self.conv(x)

In [ ]:
class EfficientNet(nn.Module):
    def __init__(self, version, num_classes):
        super(EfficientNet, self).__init__()
        width_factor, depth_factor, dropout_rate = self.calculate_factors(version)
        last_channels = ceil(1280 * width_factor)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.features = self.create_features(width_factor, depth_factor, last_channels)
        self.classifier = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(last_channels, num_classes),
        )

    def calculate_factors(self, version, alpha=1.2, beta=1.1):
        phi, res, drop_rate = phi_values[version]
        depth_factor = alpha ** phi
        width_factor = beta ** phi
        return width_factor, depth_factor, drop_rate

    def create_features(self, width_factor, depth_factor, last_channels):
        channels = int(32 * width_factor)
        features = [CNNBlock(3, channels, 3, stride=2, padding=1)]
        in_channels = channels

        for expand_ratio, channels, repeats, stride, kernel_size in base_model:
            out_channels = 4*ceil(int(channels*width_factor) / 4)
            layers_repeats = ceil(repeats * depth_factor)

            for layer in range(layers_repeats):
                features.append(
                    InvertedResidualBlock(
                        in_channels,
                        out_channels,
                        expand_ratio=expand_ratio,
                        stride = stride if layer == 0 else 1,
                        kernel_size=kernel_size,
                        padding=kernel_size//2, # if k=1:pad=0, k=3:pad=1, k=5:pad=2
                    )
                )
                in_channels = out_channels

        features.append(
            CNNBlock(in_channels, last_channels, kernel_size=1, stride=1, padding=0)
        )

        return nn.Sequential(*features)

    def forward(self, x):
        x = self.pool(self.features(x))
        return self.classifier(x.view(x.shape[0], -1))

In [ ]:
# class Identity(nn.Module):
#     def __init__(self):
#         super(Identity, self).__init__()
    
#     def forward(self, x):
#         return x

In [ ]:
version = hyper_parameter['version']
model = EfficientNet(version=version, num_classes=hyper_parameter['num_classes'])

In [ ]:
# model = models.efficientnet_b0(weights='DEFAULT')
# model.classifier = nn.Sequential(
#     nn.Dropout(p=0.2, inplace=True),
#     nn.Linear(in_features=1280, out_features=640, bias=True),
#     nn.ReLU(),
#     nn.Linear(in_features=640, out_features=hyper_parameter['num_classes'], bias=True)
#     )
# model.classifier

In [ ]:
# print(model)

In [ ]:
# define model and optimizers
model.to(device)

next(model.parameters()).device

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = hyper_parameter['learning_rate'])

# scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)
reduce_on_plateau_scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.1, patience=4, verbose=True)

In [ ]:
# view model architecture
# print(model.parameters)

In [ ]:
train_results = {
    'accuracy': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'balanced_accuracy': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'f1_score': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'loss': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'cf_matrix': np.zeros((hyper_parameter['num_classes'], hyper_parameter['num_classes']), dtype=float)
}

test_results = {
    'accuracy': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'balanced_accuracy': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'f1_score': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'loss': np.zeros(hyper_parameter['no_epochs'], dtype=float),
    'cf_matrix': np.zeros((hyper_parameter['num_classes'], hyper_parameter['num_classes']), dtype=float)
}

results = {
    'train': train_results,
    'test': test_results
}

In [ ]:
save_path = f'{checkpoint_path}.pth.tar'
print(f'Save Path: {save_path}')

def save_checkpoint(state, epoch, filename = save_path):
    print(f'=> Checkpoint at {epoch + 1} saved!')
    print(f'Saved at: {save_path}')
    torch.save(state, filename)

def load_checkpoint(checkpoint):
    print(f'=> Loading Checkpoint')
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])

In [ ]:
def train_model(model, cirterion, optimizer, scheduler, num_epochs = 1):
  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0
  best_f1_scr = 0.0
  # initialize the early stopping counter
  early_stopping_counter = 0
  early_stopping_threshold = 10
  best_test_loss = float('inf')

  epoch_preds = {}
  epoch_targets = {}

  for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch + 1, num_epochs))
    print('-' * 15)

    for phase in ['train', 'test']:
      if phase =='train':
        model.train()
      else:
        model.eval()
      
      running_loss = 0.0
      running_corrects = 0.0

      real_targets = []
      predicted_targets = []

      # Iterate over data.
      for inputs, labels in dataloaders[phase]:
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        # track history only if train
        with torch.set_grad_enabled(phase == 'train'):
          outputs = model(inputs)
          _, preds = torch.max(outputs, 1)
          
          real_targets.extend(labels.detach().cpu().numpy())
          predicted_targets.extend(preds.detach().cpu().numpy())

          loss = cirterion(outputs, labels)

          # backward + optimize only if in train
          if phase == 'train':
            loss.backward()
            optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)
        
      ### EPOCH Train & Test ###
      # if phase == 'train':
      #   scheduler.step()

      epoch_f1_score = f1_score(real_targets, predicted_targets, average='micro')
      epoch_balanced_acc = balanced_accuracy_score(real_targets, predicted_targets)
      epoch_cf_matrix = confusion_matrix(real_targets, predicted_targets)

      epoch_loss = running_loss / dataset_sizes[phase]
      epoch_acc = running_corrects / dataset_sizes[phase]

      if phase == 'test':
        epoch_preds[epoch] = predicted_targets
        epoch_targets[epoch] = real_targets
        scheduler.step()

      results[phase]['accuracy'][epoch] = epoch_acc
      results[phase]['balanced_accuracy'][epoch] = epoch_balanced_acc
      results[phase]['loss'][epoch] = epoch_loss
      results[phase]['f1_score'][epoch] = epoch_f1_score
      results[phase]['cf_matrix'] += epoch_cf_matrix

      # printing epoch resutlts
      if (epoch + 1) % 2 == 0:
        print(f'{phase} Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')
        print(f'{phase} Balanced Acc: {epoch_balanced_acc:.4f}, F1: {epoch_f1_score:.4f}')

      epoch_time = time.time() - since
      print(f'Epoch time: {epoch_time // 60:.0f}m {epoch_time %  60:.0f}s')

      if phase == 'test' and epoch_acc > best_acc:
        best_acc = epoch_acc
        best_model_wts = copy.deepcopy(model.state_dict())
      
      if phase == 'test' and epoch_f1_score > best_f1_scr:
        best_f1_scr = epoch_f1_score
      
      if epoch_loss < best_test_loss or epoch_acc > best_acc:
        best_test_loss = epoch_loss
        best_acc = epoch_acc
        early_stopping_counter = 0
      else:
        early_stopping_counter += 1
      # check if the early stopping threshold has been reached
      # if early_stopping_counter >= early_stopping_threshold:
    
    if (epoch + 1) % 2 == 0:
      checkpoint = {
        'state_dict': model.state_dict(),
        'optimizer': optimizer.state_dict()
      }
      save_checkpoint(checkpoint, epoch)

    print()
    ######################################################

  # training Complete
  # printing time require to train model
  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed // 60:.0f}m { time_elapsed % 60:.0f}s')

  t = time.localtime()
  current_time = time.strftime("%H:%M", t)
  print(f'Trainig completed at {current_time}')
  print(f'Best test Acc: {best_acc:4f}')
  print(f'Best F1 : {best_f1_scr:4f}')
  print(f"Balance Acc F1 : {results['test']['balanced_accuracy'][num_epochs-1]:4f}")

  # normalize cf matrix
  for phase in ['train', 'test']:
    cf_matrix_normalized = results[phase]['cf_matrix'] / results[phase]['cf_matrix'].sum(axis=1, keepdims=True)
    df_cm = pd.DataFrame(cf_matrix_normalized, index = class_names, columns = class_names)
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cm, annot=True)
    plt.tight_layout()
  plt.savefig(f'{checkpoint_path}_cf_matrix.svg')
  plt.show()

  for phase in ['train', 'test']:
    plt.plot(results[phase]['accuracy'], label='{} accuracy'.format(phase))
    plt.title(f'{hyper_parameter["model"]} {hyper_parameter["version"]} learning rate: {hyper_parameter["learning_rate"]} epoch: {hyper_parameter["no_epochs"]}')
    plt.legend(bbox_to_anchor = (1.45, 1), loc='upper right')
    plt.tight_layout()
  plt.savefig(f'{checkpoint_path}_accuracy.svg')
  plt.show()

  for phase in ['train', 'test']:
    plt.plot(results[phase]['balanced_accuracy'], label='{} balanced accuracy'.format(phase))
    plt.title(f'{hyper_parameter["model"]} {hyper_parameter["version"]} learning rate: {hyper_parameter["learning_rate"]} epoch: {hyper_parameter["no_epochs"]}')
    plt.legend(bbox_to_anchor = (1.45, 1), loc='upper right')
    plt.tight_layout()
  plt.savefig(f'{checkpoint_path}_balanced-accuracy.svg')
  plt.show()

  for phase in ['train', 'test']:
    plt.plot(results[phase]['f1_score'], label='{} f1 score'.format(phase))
    plt.legend(bbox_to_anchor = (1.45, 1), loc='upper right')
    plt.tight_layout()
  plt.savefig(f'{checkpoint_path}_f1_score.svg')
  plt.show()
  
  for phase in ['train', 'test']:
    plt.plot(results[phase]['loss'], label='{} loss'.format(phase))
    plt.title(f'{hyper_parameter["model"]} {hyper_parameter["version"]} learning rate: {hyper_parameter["learning_rate"]} epoch: {hyper_parameter["no_epochs"]}')
    plt.legend(bbox_to_anchor = (1.45, 1), loc='upper right')
    plt.tight_layout()
  plt.savefig(f'{checkpoint_path}_loss.svg')
  plt.show()

  last_epoch = num_epochs - 1
  preds = epoch_preds[last_epoch]
  targets = epoch_targets[last_epoch]
  print('Test Classification Report')
  print(classification_report(targets, preds))

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

In [ ]:
# load_path = f'checkpoints/efficientnet/efficientnetb0-custom_sampler-loader-full_dataset.pth.tar'
# if hyper_parameter['load_model']:
#     load_checkpoint(torch.load(load_path))
# load_path

In [ ]:
# weights = dict()
# for name, para in model.named_parameters():
#     weights[name] = para
# print(weights)

In [ ]:
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=hyper_parameter['no_epochs'])

In [ ]:
# Export to CSV
for phase in ['train', 'test']:
    for metric in ['accuracy', 'loss']:
        df = pd.DataFrame(results[phase][metric])
        df.to_csv(f'E:/CSE499/Skin-Lesion-Classificaiton/output/{hyper_parameter["model"]}-{hyper_parameter["version"]}-{phase}-{metric}-{hyper_parameter["attempt"]}.csv', mode='a')

In [ ]:
for phase in ['train', 'test']:
    csv_path = f'./output/{hyper_parameter["model"]}-{hyper_parameter["version"]}-{phase}-{metric}-{hyper_parameter["attempt"]}.csv'
    for metric in ['accuracy', 'loss']:
        prev_result = pd.read_csv(csv_path)

In [ ]:
for phase in ['train', 'test']:
    plt.plot(results[phase]['accuracy'], label='{} accuracy'.format(phase))
    plt.title(f'{hyper_parameter["model"]} {hyper_parameter["version"]} learning rate: {hyper_parameter["learning_rate"]} epoch: {hyper_parameter["no_epochs"]}')
    plt.legend(bbox_to_anchor = (1.45, 1), loc='upper right')
    plt.tight_layout()
    plt.savefig(f'accuracy.jpeg')
plt.show()

for phase in ['train', 'test']:
    df_cm = pd.DataFrame(results[phase]['cf_matrix'], index = class_names, columns = class_names)
    plt.figure(figsize = (12,7))
    sn.heatmap(df_cm, annot=True)
    plt.tight_layout()
    plt.savefig(f'cf_matrix{phase}.jpeg')
plt.show()

# Methods to try
https://stackoverflow.com/questions/62319228/number-of-instances-per-class-in-pytorch-dataset